<a href="https://colab.research.google.com/github/Nech-C/comp5300-project/blob/main/5530project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



import lib, mount google drive, clone comp5300, etc.



In [ ]:
path = "/content/drive/My Drive/GQA/"
# import lib
import os
import sys
import json
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import pandas as pd
from tqdm import tqdm
from google.colab import drive


# This will prompt for authorization.
drive.mount('/content/drive')
# Replace 'My Drive' with the specific path where you want to save the files in your Drive if needed

# Create the directory if it doesn't exist
%cd "$path"

!git -C "/content/drive/My Drive/GQA/comp5300-project" pull

sys.path.append(os.path.join(path, "comp5300-project"))
from utils.dataset_processing import *

In [ ]:
# constants - google drive
SCRIPT_PATH = "/content/drive/MyDrive/GQA/comp5300-project"
IMAGE_ZIP = '/content/drive/MyDrive/GQA/images.zip'
QUESTIONS_ZIP = '/content/drive/MyDrive/GQA/questions.zip'
SCENE_GRAPHS_ZIP = '/content/drive/MyDrive/GQA/sceneGraphs.zip'

# constants - vm
BASE = '/content/gqa'

!cp $IMAGE_ZIP $BASE/images
!cp $QUESTIONS_ZIP $BASE/questions
!cp $SCENE_GRAPHS_ZIP $BASE/sceneGraphs
# create dirs and unzip those in BASE
!mkdir -p $BASE/images
!mkdir -p $BASE/questions
!mkdir -p $BASE/sceneGraphs
!unzip -q $QUESTIONS_ZIP -d $BASE/questions
!unzip -q $SCENE_GRAPHS_ZIP -d $BASE/sceneGraphs
!unzip -q $IMAGE_ZIP -d $BASE/images

cp: cannot create regular file '/content/gqa/images': No such file or directory
cp: cannot create regular file '/content/gqa/questions': No such file or directory
cp: cannot create regular file '/content/gqa/sceneGraphs': No such file or directory


In [ ]:
# preprocess questions

import os
import json

TRAIN_ALL_QUESTIONS = "/content/gqa/questions/train_all_questions"
IMAGES = "/content/gqa/images/images"
PROCESSED_QUESTIONS = "/content/drive/MyDrive/GQA/processed_questions"

# Create the processed_questions directory if it doesn't exist
os.makedirs(PROCESSED_QUESTIONS, exist_ok=True)

# Get a list of all the image names in the images directory
image_names = set(os.listdir(IMAGES))

# Get a list of all the JSON files in the train_all_questions directory
json_files = [f for f in os.listdir(TRAIN_ALL_QUESTIONS) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(TRAIN_ALL_QUESTIONS, json_file)

    with open(json_path, 'r') as f:
        questions_data = json.load(f)

    processed_questions = {}

    for question_id, question_info in questions_data.items():
        image_id = question_info['imageId']
        image_name = image_id + '.jpg'

        # Check if the image exists in the set of image names
        if image_name in image_names:
            processed_question = {
                'imageId': image_id,
                'question': question_info['question'],
                'answer': question_info['answer'],
                'fullAnswer': question_info['fullAnswer'],
                'groups': question_info['groups']
            }
            processed_questions[question_id] = processed_question
    # Save the processed questions to a new JSON file
    processed_json_path = os.path.join(PROCESSED_QUESTIONS, json_file)
    with open(processed_json_path, 'w') as f:
        json.dump(processed_questions, f, indent=2)

print("Processing completed.")

Processing completed.


In [ ]:
# create trainable .jsons
TRAIN_ALL_QUESTIONS = "/content/gqa/questions/train_all_questions"
IMAGES = "/content/gqa/images/images"
PROCESSED_QUESTIONS = "/content/drive/MyDrive/GQA/processed_questions"
processed_json_files = [os.path.join(PROCESSED_QUESTIONS, f) for f in os.listdir(PROCESSED_QUESTIONS) if f.endswith('.json')]
trainable_save_path = "/content/drive/MyDrive/GQA/trainable_questions"
os.makedirs(trainable_save_path, exist_ok=True)

for json_file in processed_json_files:
    with open(json_file, 'r') as f:
        questions_data = json.load(f)

        # generate trainable .json
        trainable_questions = []
        for question_id, question_info in questions_data.items():
            image_name = question_info['imageId'] + '.jpg'

            trainable_question = {
              'id': question_id,
              'image': image_name,
              "conversations": [
                  {
                      "from": "human",
                      "value": question_info['question']
                  },
                  {
                      "from": "gpt",
                      "value": question_info['answer']
                  }
              ],
            }
            trainable_questions.append(trainable_question)
            # end of for

        trainable_json_path = os.path.join(
            trainable_save_path,
            os.path.splitext(os.path.basename(json_file))[0] + "_trainable.json")

        with open(trainable_json_path, 'w') as f:
            json.dump(trainable_questions, f, indent=2)

print("Processing completed.")


Processing completed.


NEW START POINT

In [ ]:
import os
import sys
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# copy trainables to disk
!pip install deepspeed
!git lfs install
!git clone https://huggingface.co/liuhaotian/llava-v1.6-mistral-7b
!git clone https://github.com/bdytx5/finetune_LLaVA

In [ ]:
IMAGE_ZIP = '/content/drive/MyDrive/GQA/images.zip'
QUESTIONS_ZIP = '/content/drive/MyDrive/GQA/questions.zip'
TRAINABLE_PATH = '/content/drive/MyDrive/GQA/trainable_questions'

# constants - vm
BASE = '/content/datasets'

!cp $IMAGE_ZIP /content
# create dirs and unzip those in BASE
!mkdir -p $BASE/images
!unzip -q /content/images.zip -d $BASE/images
!cp -r $TRAINABLE_PATH $BASE

cp: cannot create regular file '/content/datasets/images': No such file or directory


In [ ]:
!pip install --upgrade pip
%cd /content/finetune_LLaVA
!pip install -e .
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation

In [ ]:
import json
import random
with open('/content/datasets/trainable_questions/train_all_questions_0_trainable.json', 'r') as f:
  data = json.load(f)
  # get random 6000 items and save to /content/datasets/trainable_questions/train_questions_trainable.json
  random_items = random.sample(data, 10000)
  with open('/content/datasets/trainable_questions/train_questions_trainable.json', 'w') as f:
    json.dump(random_items, f, indent=2)

In [ ]:
with open('/content/datasets/trainable_questions/train_all_questions_1_trainable.json', 'r') as f:
  data = json.load(f)
  # get random 3000 items save to /content/datasets/trainable_questions/val_questions.json
  random_items = random.sample(data, 3000)
  with open('/content/datasets/trainable_questions/val_questions.json', 'w') as f:
    json.dump(random_items, f, indent=2)

In [ ]:
sys.path.append('/content/LLaVA/llava')
sys.path.append('/content/LLaVA')
!deepspeed /content/finetune_LLaVA/llava/train/train_mem.py \
--deepspeed /content/finetune_LLaVA/scripts/zero2.json \
--lora_enable True \
--lora_r 128 \
--lora_alpha 256 \
--mm_projector_lr 2e-5 \
--bits 4 \
--model_name_or_path /content/llava-v1.6-mistral-7b \
--version llava_llama_1.6 \
--data_path /content/datasets/trainable_questions/train_questions_trainable.json \
--validation_data_path /content/datasets/trainable_questions/val_questions.json \
--image_folder /content/datasets/images/images\
--vision_tower openai/clip-vit-large-patch14-336 \
--mm_projector_type mlp2x_gelu \
--mm_vision_select_layer -2 \
--mm_use_im_start_end False \
--mm_use_im_patch_token False \
--image_aspect_ratio pad \
--group_by_modality_length True \
--bf16 True \
--output_dir /content/llama-1.6-7b-chat-task-qlora \
--num_train_epochs 20 \
--per_device_train_batch_size 256 \
--per_device_eval_batch_size 256 \
--gradient_accumulation_steps 1 \
--evaluation_strategy "epoch" \
--save_strategy "steps" \
--save_steps 500 \
--save_total_limit 1 \
--learning_rate 2e-4 \
--weight_decay 0. \
--warmup_ratio 0.03 \
--lr_scheduler_type "cosine" \
--logging_steps 1 \
--tf32 True \
--model_max_length 2048 \
--gradient_checkpointing True \
--dataloader_num_workers 2 \
--lazy_preprocess True \
--report_to wandb

evaluate fine tuned model:

In [ ]:
# clone required repos
!pip install deepspeed
!git lfs install
!git clone https://huggingface.co/liuhaotian/llava-v1.6-mistral-7b
!git clone https://github.com/bdytx5/finetune_LLaVA

# instal dependecies
!pip install --upgrade pip
%cd /content/finetune_LLaVA
!pip install -e .
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation

# copy files